In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import Data for V1.0 Feature Selection

In [16]:
X_train= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.0/x_Train.csv')
X_test= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.0/x_Test.csv')
y_train= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.0/y_Train.csv')
y_test= pd.read_csv('C:/Users/jmeis/NYC_DSA/HousingPrices/Data/Version_1.0/y_Test.csv')

# Create Basic Linear Regression for comparison Purposes

In [19]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(solver='liblinear')
logit.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [20]:
logit.score(X_train, y_train)

0.9841427155599604

## Make Predicitions 

In [28]:
PredLin = pd.concat([pd.Series(logit.predict(X_test)), y_test], axis = 1)
PredLin.columns = ['Predicted', 'Actual']
PredLin


,Predicted,Actual
0,244000,136000
1,174000,194201
2,287000,294000
3,167000,132000
4,128000,119000
...,...,...
428,309000,274725
429,97000,84000
430,361919,485000
431,185000,215000


## Let's Score Ourselves

In [35]:
PredLin['Difference']=(PredLin['Predicted'] - PredLin['Actual'])**2
RSS = PredLin.Difference.sum()
RSS

1287134913755

In [38]:
Average = PredLin.Difference.mean()**.5
Average

54521.536600180334

In [39]:
logit.score(X_test, y_test)

0.006928406466512702

**Yeah.......that's bad.....**

# Fit an ElasticNet Model With GridSearch

In [41]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
parametersGrid = {"max_iter": [1, 5, 10],
                "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                "l1_ratio": np.arange(0.0, 1.0, 0.1)}
eNet = ElasticNet()
grid = GridSearchCV(eNet, parametersGrid, scoring='r2', cv=10)
grid.fit(X_train, y_train)
pred = grid.predict(X_test)
    

In [43]:
PredGrid = pd.concat([pd.Series(pred), y_test], axis = 1)
PredGrid.columns = ['Predicted', 'Actual']
PredGrid

,Predicted,Actual
0,131303.775432,136000
1,228804.665819,194201
2,259298.482571,294000
3,159068.195721,132000
4,103835.620502,119000
...,...,...
428,244651.207131,274725
429,60714.823459,84000
430,427173.293038,485000
431,214172.771764,215000


## Lets score ourselves

In [44]:
PredGrid['Difference']=(PredGrid['Predicted'] - PredGrid['Actual'])**2
PredGrid.Difference.sum()


319799827969.89417

In [45]:
PredGrid.Difference.mean()**.5


27176.60265802347

In [46]:
grid.score(X_test, y_test)

0.8634328446890307

In [47]:
grid.get_params()

{'cv': 10,
 'error_score': nan,
 'estimator__alpha': 1.0,
 'estimator__copy_X': True,
 'estimator__fit_intercept': True,
 'estimator__l1_ratio': 0.5,
 'estimator__max_iter': 1000,
 'estimator__normalize': 'deprecated',
 'estimator__positive': False,
 'estimator__precompute': False,
 'estimator__random_state': None,
 'estimator__selection': 'cyclic',
 'estimator__tol': 0.0001,
 'estimator__warm_start': False,
 'estimator': ElasticNet(),
 'n_jobs': None,
 'param_grid': {'max_iter': [1, 5, 10],
  'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
  'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': 'r2',
 'verbose': 0}

In [67]:
fig = px.scatter(PredGrid, x = 'Predicted', y = 'Actual', title = 'Predicted Vs Actual House Prices from V1.0 Model')
fig

In [66]:
V1_0Coef = pd.concat( [pd.Series(X_train.columns),pd.Series(grid.best_estimator_.coef_)] , axis = 1)
V1_0Coef.columns = ['Feature', 'Coefficient']
pd.options.display.max_rows = 999
V1_0Coef


,Feature,Coefficient
0,MSSubClass,-115.338013
1,LotFrontage,-42.418466
2,LotArea,0.277639
3,YearBuilt,154.808713
4,YearRemodAdd,305.426459
5,MasVnrArea,45.451509
6,BsmtFinSF1,38.816086
7,BsmtFinSF2,15.120310
8,BsmtUnfSF,14.733089
9,LowQualFinSF,-41.057512


In [68]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [69]:
vif_data = pd.DataFrame()
vif_data['Feature'] = X_train.columns
vif_data['VIF'] = [vif(X_train.values, i) for i in range(len(X_train.columns))]

C:\Users\jmeis\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning:

divide by zero encountered in double_scalars

C:\Users\jmeis\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning:

invalid value encountered in double_scalars



In [70]:
vif_data

,Feature,VIF
0,MSSubClass,41.056233
1,LotFrontage,14.449652
2,LotArea,17.119041
3,YearBuilt,16.271130
4,YearRemodAdd,3.842474
5,MasVnrArea,3.367215
6,BsmtFinSF1,33.596829
7,BsmtFinSF2,9.345433
8,BsmtUnfSF,33.098595
9,LowQualFinSF,2.709805
